# 1 - Exploration et nettoyage

In [ ]:
import pandas as pd
pd.set_option("display.max_colwidth", None)

In [ ]:
# Création du DataFrame à partir du fichier CSV
df = pd.read_csv('flickr_data2.csv')

In [ ]:
df.info()

On enlève les espaces en début de nom de colonne.

In [ ]:
df.columns = df.columns.str.strip()

## Colonnes de dates

Les colonnes de dates sont converties en numérique.

In [ ]:
date_cols = [
    "date_taken_minute", "date_taken_hour", "date_taken_day",
    "date_taken_month", "date_taken_year",
    "date_upload_minute", "date_upload_hour", "date_upload_day",
    "date_upload_month", "date_upload_year",
]

for c in date_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

On s'intéresse à la cohérence de ces colonnes.

In [ ]:
date_rules = {
    "date_taken_minute": (0, 59),
    "date_taken_hour": (0, 23),
    "date_taken_day": (1, 31),
    "date_taken_month": (1, 12),
    "date_taken_year": (1900, 2026),

    "date_upload_minute": (0, 59),
    "date_upload_hour": (0, 23),
    "date_upload_day": (1, 31),
    "date_upload_month": (1, 12),
    "date_upload_year": (1900, 2026),
}

outlier_mask = pd.Series(False, index=df.index)

for col, (lo, hi) in date_rules.items():
    outlier_mask |= ~df[col].between(lo, hi)

df_outliers = df[outlier_mask]
df_outliers.head(5)

Plusieurs colonnes contiennent des valeurs aberrantes (trop petites ou trop grandes) qui ne correspondent pas à des dates valides.
On décide de créer de nouvelles colonnes de type datetime : les lignes avec des valeurs aberrantes seront converties en NaT (Not a Time).

Si on a besoin de faire des analyses sur la date, on les fera en utilisant ces nouvelles colonnes nettoyées.
On ne supprime pas les lignes avec des valeurs aberrantes, car elles peuvent contenir des informations utiles dans d'autres colonnes.

In [ ]:
df["taken_dt"] = pd.to_datetime(
    dict(
        year=df["date_taken_year"],
        month=df["date_taken_month"],
        day=df["date_taken_day"],
        hour=df["date_taken_hour"],
        minute=df["date_taken_minute"],
    ),
    errors="coerce"
)

df["upload_dt"] = pd.to_datetime(
    dict(
        year=df["date_upload_year"],
        month=df["date_upload_month"],
        day=df["date_upload_day"],
        hour=df["date_upload_hour"],
        minute=df["date_upload_minute"],
    ),
    errors="coerce"
)

df[["taken_dt", "upload_dt"]].info()

## Colonnes Unnamed

On décide de supprimer les colonnes inutiles "Unnamed: 16", "Unnamed: 17" et "Unnamed: 18" car elles ne contiennent aucune information pertinente pour notre analyse, et sont globalement vides.

In [ ]:
df.drop(columns=['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18'], inplace=True)
df.info()

## Lignes dupliquées

On vient vérifier le nombre de lignes dupliquées dans le DataFrame.

In [ ]:
n_row_dupes = df.duplicated().sum()
n_row_dupes

Il y a 252 139 lignes dupliquées dans le DataFrame. On les supprime.

In [ ]:
df = df.drop_duplicates()
df.info()

## Colonne URL

On ajoute la colonne "url" qui sera utile pour accéder aux images.

In [ ]:
df["url"] = (
    "https://www.flickr.com/photos/"
    + df["user"].astype(str)
    + "/"
    + df["id"].astype(str)
)
df.info()

## Colonnes latitude, longitude

On vérifie la cohérence des colonnes "latitude" et "longitude".

In [ ]:
geo_rules = {
    "lat": (-90, 90),
    "long": (-180, 180)
}

outlier_mask = pd.Series(False, index=df.index)
for col, (lo, hi) in geo_rules.items():
    outlier_mask |= ~df[col].between(lo, hi)

df_geo_outliers = df[outlier_mask]
df_geo_outliers.shape

Pas de problème apparent.

In [ ]:
df.to_csv("flickr_data_cleaned.csv", index=False)

In [ ]:
df

## Colonnes title, tags

In [ ]:
for c in ["title", "tags", "user"]:
    df[c] = df[c].astype("string")
    df[c] = df[c].fillna("").str.strip()
df["title"] = df["title"].str.replace(r"\s+", " ", regex=True)

On transforme des tags en liste.

In [ ]:
def split_tags(s: str):
    if not isinstance(s, str) or s.strip() == "":
        return []
    items = [t.strip().lower() for t in s.split(",")]
    items = [t for t in items if t]            # vire vides
    items = sorted(set(items))                 # dédoublonne
    return items

df["tag_list"] = df["tags"].map(split_tags)
df["tag_count"] = df["tag_list"].map(len)

In [ ]:
df

In [ ]:
df.to_csv("flickr_data_cleaned.csv", index=False)

In [ ]:
df.info()